# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [ ]:
class EncoderDecoder(nn.Module):
  ''' 
  Transformer는 Encoder-Decoder 모델을 따르며, 주요 architecture로는 stacked 
  self-sttention, point-wise fully connected layer를 사용
  '''
    
  def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
    super(EncoderDecoder, self).__init__()      
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.generator = generator
    
    
  def forward(self, src, tgt, src_mask, tgt_mask):
    '''masked src & target sequence를 받는다'''
    return self.decode(self.encoder(src, src_mask), src_mask, tgt, tgt_mask)
    
    
  def encode(self, src, src_mask):
    return self.encoder(self.src_embed(src), src_mask)
    
    
  def decode(self, memory, src_mask, tgt, tgt_mask):
    return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
  ''' Standard Linear 와 softmax generation step 정의'''
  def __init__(self, d_model, vocab):
      super(Generator, self).__init__()
      self.proj = nn.Linear(d_model, vocab)
    
    
  def forward(self, x):
    return F.log_softmax(self.proj(x), dim=-1)

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [ ]:
def clones(module, N):
  '''N개의 동일한 layer를 만든다.'''
  # nn.Module을 리스트로 정리하는 방법 :
  # 각 레이어를 리스트에 전달하고 레이어 iterator 를 만드는것 
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
  '''Core Encoder는 6개 layer 층이 쌓여있다'''
  def __init__(self, layer, N):
    super(Encoder, self).__init__()
    self.layers = clones(layer, N) # nn.ModuleList([copy.deepcopy(layer) for _ in range(6)])
    self.norm = LayerNorm(layer.size) # layer normalization layer를 추가하여 성능을 높인다
    
  def forward(self, x, mask):
    ''' 각 레이어 순서대로 입력 값을 넘기며 mask를 씌운다'''
    for layer in self.layers: # layers 들 중 each layer 
      x = layer(x, mask)
    return self.norm(x)

In [ ]:
class LayerNorm(nn.Module):
  
  '''
  2개의 sub-layer에 layer norm 적용
  #layernorm : 각 input의 feature들에 대한 평균과 분산을 구해 batch에 있는 각 input을 정규화
  '''
  def __init__(self, features, eps=1e-6):
    super(LayerNorm, self).__init__() 
    self.a_2 = nn.Parameter(torch.ones(features)) #features 수 만큼 '1'로 채움
    self.b_2 = nn.Parameter(torch.zeros(features)) #features 수 만큼 '0'로 채움
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection(nn.Module):
  
  '''
  layernorm 적용 이후 residual connection 적용
  residual connection : 기존에 학습한 정보를 보존하고, 거기에 추가적으로 학습하는 정보 의미 
  eg. 오픈북이 가능한 시험 
  '''
  def __init__(self, size, dropout):
    super(SublayerConnection, self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, sublayer):
    '''동일 사이즈의 sublayer에 residual connection 적용'''
    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer(nn.Module):
  '''
  Encoder는 self attention과 feed forward로 구성 
  '''
  def __init__(self, size, self_attn, feed_forward, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size, dropout), 2)
    self.size = size

  def forward(self, x, mask):
    ''' Transformer Architecture의 왼쪽 부분'''
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) #multi-head attention + add & norm
    return self.sublayer[1](x, self.feed_forward) #feed_forward + add & norm

### Decoder
- `Decoder`
- `DecoderLayer`

In [ ]:
class Decoder(nn.Module):
    
    '''
    Decoder 또한 6개의 layer
    '''
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        ''' 각 레이어 순서대로 입력 값을 넘기며 mask를 씌움'''
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask) #memory key, value= encoder의 output
        return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):

    '''
    encoder와 달리 decoder는 3번째 sublayer가 있음 
    => encoder stack의 output에 multi-head attention을 수행
    '''
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3) #3개의 sublayer
 
    def forward(self, x, memory, src_mask, tgt_mask):
        '''transformer architecture 오른쪽 부분'''
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask)) #masked_multi_head_attention + add & norm
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask)) #multi-head attention + add & norm
        return self.sublayer[2](x, self.feed_forward) #feed_forward + add & norm 

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [ ]:
   # query 크기: (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
   # key 크기: (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
   # value 크기: (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
   # padding_mask: (batch_size, 1, 1, key의 문장 길이)

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    '''Scaled Dot Product Attention연산'''
    # W-q, W_k, W_v 세개의 shape은 전부 (d_model, d_k)
    # attention 함수는 주어진 Q에 대해서 모든 K와의 유사도를 각각 구한다.
    # 이 유사도를 가중치로 하여 K와 맵핑되어 있는 각각의 V에 반영 
    # 그리고 유사도가 반영된 V을 모두 가중합하여 return 
    d_k = query.size(-1) # d_model/num_heads
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
             #Q*K = 어텐션스코어행렬
             #루트값으로 나눠준다
    if mask is not None: 
    # mask : 어텐션 스코어 행렬의 마스킹할 위치에 매우 작은 음수값을 넣어준다
    # 매우 작은 값이므로 소프트 맥스 함수를 지나면 행렬의 해당 위치 값은 0이 된다.
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    # p_attn (attention_weight) : (batch_size, num_heads, query의 문장길이, d_model/num_heads)
    if dropout is not None:
        p_attn = dropout(p_attn)
        # return[0] = (batch_size, num_heads, query문장길이, d_model/num_heads)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
#연습용 
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return scores.size(), p_attn.size(), torch.matmul(p_attn, value).size()

In [ ]:
#연습용
Q = torch.rand(12, 512)
K = Q
V = Q

attention(Q, K, V)


(torch.Size([12, 12]), torch.Size([12, 12]), torch.Size([12, 512]))

In [ ]:
# 연습용
np.set_printoptions(suppress=True)

temp_k = torch.tensor([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype = torch.float32)  # (4, 3)

temp_v = torch.tensor([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype = torch.float32)  # (4, 3)

temp_q = torch.tensor([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype = torch.float32)  # (4, 3)

attention(temp_q, temp_k, temp_v)


(torch.Size([4, 4]), torch.Size([4, 4]), torch.Size([4, 3]))

###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : (m, m)
- p_attn : (m, m)
- attention : (m, d_k)

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : torch.Size([12, 8, 1, 1])
- p_attn : torch.Size([12, 8, 1, 1])
- attention : torch.Size([12, 8, 1, 64])

In [ ]:
# 연습용

Q = torch.rand(12, 8, 1, 64)
K = Q
V = Q
nbatches = Q.size(0)

attention(Q, K, V)


(torch.Size([12, 8, 1, 1]),
 torch.Size([12, 8, 1, 1]),
 torch.Size([12, 8, 1, 64]))

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        '''모델 크기 d_model과 head의 개수를 input으로 받는다'''
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # d_v = d_k 라고 항상 가정 
        self.d_k = d_model // h #depth of model 
        self.h = h # num_heads #논문에서는 8로 지정 = 8개의 병렬 어텐션이 이루어지게 됨
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Q, K, V를 여러개의 head로 (multi head로)나눈다. 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) multi head의 attention을 구하기 위해 scaled dot production을 사용한다. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) Concat x.shape(bs, n_seq, n_head * d_head)
        # 4) linear 과정
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)   

In [ ]:
# 연습용
d_model = 512
h = 8
d_k = d_model//h
d_v = d_model//h

Q = torch.rand(12, 512)
K = Q
V = Q
nbatches = Q.size(0)

hi = nn.Linear(d_model, d_model)(Q).size()
print(hi)
hi = nn.Linear(d_model, d_model)(Q).view(nbatches, -1, h, d_k).size()
print(hi)
hi = nn.Linear(d_model, d_model)(Q).view(nbatches, -1, h, d_k).transpose(1,2).size()
print(hi)


torch.Size([12, 512])
torch.Size([12, 1, 8, 64])
torch.Size([12, 8, 1, 64])


### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : ([1, 1])
- `nn.Linear(d_model, d_model)(query)` : torch.Size([12, 512])
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : torch.Size([12, 1, 8, 64])
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : torch.Size([12, 8, 1, 64]) 

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [ ]:
class PositionwiseFeedForward(nn.Module):
    '''
    Fully connectd Feed-Forward network 구성
    - 2개의 linear layer
    - ReLu Activation function
    #FFN(x)=max(0,xW1+b1)W2+b2
    '''
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff) #inner layer의 차원은 d_ff
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
class Embeddings(nn.Module):
    '''
    input/output을 d_model로 embedding함
    linear transformation & softmax function 이용해서 
    decoder의 output을 predicted next-token probabilities로 변경

    *특이점: transformer model의 두개의 embedding layer와 이 softmax
    function의 weight이 같다. = linear transformation에서 모두 같은 weight 사용
    '''
  
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [ ]:
class PositionalEncoding(nn.Module):
    '''
    위치에 대한 정보 제공
    positional encoding 차원 = embedding 차원
    d_model, embedding vector와 더함으로써 위치 정보를 넣어준다 

    '''
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [ ]:
# 연습용
class PositionalEncoding(nn.Module):
    
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [ ]:
#연습용
position = torch.arange(0, 512).unsqueeze(1)
print(position.size())
d_model = 512
div_term = torch.exp(torch.arange(0, d_model, 2) *
     -(math.log(10000.0) / d_model))
print(div_term.size())
third = position * div_term 
print(third.size())

torch.Size([512, 1])
torch.Size([256])
torch.Size([512, 256])


### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : torch.Size([512, 1])
- `div_term` : torch.Size([256])

- `position * div_term` : torch.Size([512, 256])


### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [ ]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    '''
    hyperaparmater설정하여 full model로 만들어주는 함수  
    '''
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [ ]:
model = make_model(10,10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [ ]:
for name, param in model.named_parameters():
  print(name)
  print(param.shape)

encoder.layers.0.self_attn.linears.0.weight
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.0.bias
torch.Size([512])
encoder.layers.0.self_attn.linears.1.weight
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.1.bias
torch.Size([512])
encoder.layers.0.self_attn.linears.2.weight
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.2.bias
torch.Size([512])
encoder.layers.0.self_attn.linears.3.weight
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.3.bias
torch.Size([512])
encoder.layers.0.feed_forward.w_1.weight
torch.Size([2048, 512])
encoder.layers.0.feed_forward.w_1.bias
torch.Size([2048])
encoder.layers.0.feed_forward.w_2.weight
torch.Size([512, 2048])
encoder.layers.0.feed_forward.w_2.bias
torch.Size([512])
encoder.layers.0.sublayer.0.norm.a_2
torch.Size([512])
encoder.layers.0.sublayer.0.norm.b_2
torch.Size([512])
encoder.layers.0.sublayer.1.norm.a_2
torch.Size([512])
encoder.layers.0.sublayer.1.norm.b_2
torch.Size([512])
encoder.layers.1.self_attn.

In [ ]:
# 구현
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)

+---------------------------------------------+------------+
|                   Modules                   | Parameters |
+---------------------------------------------+------------+
| encoder.layers.0.self_attn.linears.0.weight |   262144   |
|  encoder.layers.0.self_attn.linears.0.bias  |    512     |
| encoder.layers.0.self_attn.linears.1.weight |   262144   |
|  encoder.layers.0.self_attn.linears.1.bias  |    512     |
| encoder.layers.0.self_attn.linears.2.weight |   262144   |
|  encoder.layers.0.self_attn.linears.2.bias  |    512     |
| encoder.layers.0.self_attn.linears.3.weight |   262144   |
|  encoder.layers.0.self_attn.linears.3.bias  |    512     |
|   encoder.layers.0.feed_forward.w_1.weight  |  1048576   |
|    encoder.layers.0.feed_forward.w_1.bias   |    2048    |
|   encoder.layers.0.feed_forward.w_2.weight  |  1048576   |
|    encoder.layers.0.feed_forward.w_2.bias   |    512     |
|     encoder.layers.0.sublayer.0.norm.a_2    |    512     |
|     encoder.layers.0.s

44155914